Loading data from csv

In [ ]:
import pandas as pd
data = pd.read_csv('../data/cleanedHouses.csv', sep=',')

training model and saving the results to find best combination later

In [ ]:
from sklearn.model_selection import train_test_split
import itertools
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

input_features = ["usable_area","land_area","garage","parkingLots","region","district","room_count","condition"]
target_feature = "price"

combinationsScores = {}
for loop in range(1,10+1):
    for r in range(5, len(input_features) + 1):
        for combo in itertools.combinations(input_features, r):
            X_train, X_test, y_train, y_test = train_test_split(data[list(combo)], data[target_feature], test_size=0.25)
            model = GradientBoostingRegressor(n_estimators=100)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            if combo not in combinationsScores.keys():
                combinationsScores[combo] = []
            combinationsScores[combo].append(mse**(1/2))
            print(f"{loop} | Done: {combo}")


Finding the best model by attributes avg

In [ ]:
import json

bestAvg = None
bestCombination = None

tempcombo = {}

for key,val in combinationsScores.items():
    idk = str(key)
    tempcombo[idk] = val

with open("../SavedModels/GradientBoostingRegressorData.json","w") as f:
    json.dump(tempcombo,f,indent=4)

for key,val in combinationsScores.items():
    score = 0
    for i in val:
        score += i
    avg = score/len(val)
    if (bestAvg is None or avg < bestAvg):
        bestAvg = avg
        bestCombination = key

print("Best")
print(f"Combination: {bestCombination}")
print(f"Average: {bestAvg}")

Training the model based on the best attr

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

target_feature = "price"

X_train, X_test, y_train, y_test = train_test_split(data[list(bestCombination)], data[target_feature], test_size=0.1)
model = LinearRegression(n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

Saving model to dat for later use

In [ ]:
import pickle
import json

with open("../SavedModels/GradientBoostingRegressor.dat","wb") as f:
    pickle.dump(model,f)

with open("../SavedModels/GradientBoostingRegressor.json","w") as f:
    json.dump(bestCombination,f)